QUERYING THE IMDB DATABASE

Due by 23rd of December 2020, 17:00


You are required to write 4 SQL SELECT statements on the IMDB dataset using the binder repo satisfying the following conditions:

- At least one of the queries should incorporate a single or multiple "WITH" (CTE) statements(s)
- At least one of the queries should incorporate a subquery
- At least one of the queries should have a GROUP BY and HAVING caluses
- At least two of the queries should incorporate JOINS (LEFT, RIGHT, INNER, FULL) of two or more tables
- All queries should have at least two logical conditions in WHERE clauses (note that the logical conditions can be in any part of the statement - WITH queries, subqueries, etc - and can be in separate parts - e.g. one in WITH query, the other in the main query)
- All queries should have at least one ordering field with an ORDER BY clause in the main statement
- Each query should return at most 20 records (you can use LIMIT for that)
- The purpose or rationale of the query (what the query is meant to do) should be provided clearly in English before each query 
- The output of each query should be provided along


You can provide more than 4 queries (however, not more than 6) if you think you cannot satisfy the above conditions with 4 queries


You will submit one simple TEXT FILE that includes:
- intention of the query
- the query itself
- the output of the query

# Preparation 

Starting postgresql from terminal

In [1]:
sudo service postgresql start

Starting PostgreSQL 11 database server: main.


Connecting to database

In [2]:
%load_ext sql
%sql postgres://postgres@localhost/imdb2

'Connected: postgres@imdb2'

# Query 1 - Best 20 Episodes in series

This query will return best 20 episodes in the dataset. 
It uses; 
 * Subquery 
 * Having clause
 * Multiple CTE's
 * Multiple Window Functions

In [3]:
%%sql
WITH SERIES_TOTAL_VOTE AS(
SELECT 
TE.PARENTTCONST AS SERIES_ID,
SUM(TR.AVERAGERATING*TR.NUMVOTES) AS WEIGHT,
SUM(TR.NUMVOTES) AS series_total_votes
FROM TITLE_EPISODE TE
INNER JOIN TITLE_RATINGS TR ON (TE.PARENTTCONST = TR.TCONST)
WHERE 1=1 
AND TE.TCONST <> TE.PARENTTCONST --We are eliminating anything has same content as itself so we are getting series
GROUP BY TE.PARENTTCONST
HAVING SUM(TR.NUMVOTES) >= 1000 --We used this filter to eliminate series which got didn't got voted significantly
),
SERIES_MAIN_DATA AS(
SELECT 
STV.SERIES_ID,
TB_SRS.PRIMARYTITLE AS SERIES_NAME,
TB_SRS.STARTYEAR AS SERIES_STARTING_YEAR,
TB_SRS.ENDYEAR AS SERIES_ENDING_YEAR,
(STV.WEIGHT/STV.SERIES_TOTAL_VOTES) AS SERIES_WEIGHTED_AVERAGE_RATING,
STV.SERIES_TOTAL_VOTES
FROM SERIES_TOTAL_VOTE STV
INNER JOIN TITLE_BASICS TB_SRS ON (STV.SERIES_ID = TB_SRS.TCONST)
WHERE 1 = 1 
--AND stv.series_total_votes >= 1000 --We could've used this instead of above having clause
), 
EPISODES_MAIN_DATA AS(
SELECT 
SMD.SERIES_ID,
SMD.SERIES_NAME,
SMD.SERIES_STARTING_YEAR,
SMD.SERIES_ENDING_YEAR,
TE.TCONST AS EPISODE_ID,
TB_EP.PRIMARYTITLE AS episode_name,
TE.SEASONNUMBER AS season_number,
TE.EPISODENUMBER AS episode_number,
TB_EP.STARTYEAR AS episode_year,
round(SMD.SERIES_WEIGHTED_AVERAGE_RATING, 1) AS SERIES_WEIGHTED_AVERAGE_RATING,
SMD.SERIES_TOTAL_VOTES,
TR.AVERAGERATING AS EPISODE_RATING,
TR.NUMVOTES AS EPISODE_TOTAL_VOTES,
--max(TR.AVERAGERATING) OVER (PARTITION BY SMD.SERIES_ID) AS BEST_EPISODE_RATING --We could've used this instead of subquery
(SELECT MAX(AVERAGERATING) FROM TITLE_RATINGS WHERE TCONST = TE.TCONST GROUP BY TCONST) AS BEST_EPISODE_RATING
FROM SERIES_MAIN_DATA SMD
INNER JOIN TITLE_EPISODE TE ON (SMD.SERIES_ID = TE.PARENTTCONST)
INNER JOIN TITLE_BASICS TB_EP ON (TE.TCONST = TB_EP.TCONST)
INNER JOIN TITLE_RATINGS TR ON (TE.TCONST = TR.TCONST)
WHERE 1 = 1 
AND TR.NUMVOTES > 100 --We used this filter to eliminate episodes which got didn't got voted significantly
), 
PLURAL_BY_SERIES AS (
SELECT
EMD.SERIES_ID,
EMD.SERIES_NAME,
EMD.SERIES_STARTING_YEAR,
EMD.SERIES_ENDING_YEAR,
EMD.EPISODE_ID,
EMD.EPISODE_NAME,
EMD.SEASON_NUMBER,
EMD.EPISODE_NUMBER,
EMD.EPISODE_YEAR,
EMD.SERIES_WEIGHTED_AVERAGE_RATING,
EMD.SERIES_TOTAL_VOTES,
EMD.EPISODE_RATING,
EMD.EPISODE_TOTAL_VOTES,
RANK() OVER (PARTITION BY EMD.SERIES_ID ORDER BY EMD.EPISODE_ID DESC) AS EPISODE_NO_BY_SERIES
FROM EPISODES_MAIN_DATA EMD
WHERE 1 = 1 
AND EMD.EPISODE_RATING = EMD.BEST_EPISODE_RATING --We used this filter to see only best episodes in each series
), 
ALL_BEST_BY_SERIES AS (
SELECT 
PBS.SERIES_ID,
PBS.SERIES_NAME,
PBS.SERIES_STARTING_YEAR,
PBS.SERIES_ENDING_YEAR,
PBS.EPISODE_ID,
PBS.EPISODE_NAME,
PBS.SEASON_NUMBER,
PBS.EPISODE_NUMBER,
PBS.EPISODE_YEAR,
PBS.SERIES_WEIGHTED_AVERAGE_RATING,
PBS.SERIES_TOTAL_VOTES,
PBS.EPISODE_RATING,
PBS.EPISODE_TOTAL_VOTES,
RANK() OVER (ORDER BY PBS.EPISODE_RATING DESC, PBS.SERIES_ID DESC) AS RANK_BY_VOTES,
RANK() OVER (PARTITION BY PBS.EPISODE_YEAR ORDER BY PBS.EPISODE_RATING DESC, PBS.SERIES_ID DESC) AS RANK_BY_VOTES_ANNUALLY
FROM PLURAL_BY_SERIES PBS
WHERE 1 = 1 
AND PBS.EPISODE_NO_BY_SERIES = 1 --If 2 episodes have same ratings in same series they will both show up, this one prevents that
)
SELECT
ABBS.SERIES_ID,
ABBS.SERIES_STARTING_YEAR,
ABBS.SERIES_NAME,
ABBS.SERIES_ENDING_YEAR,
ABBS.EPISODE_ID,
ABBS.EPISODE_NAME,
ABBS.SEASON_NUMBER,
ABBS.EPISODE_NUMBER,
ABBS.EPISODE_YEAR,
ABBS.SERIES_WEIGHTED_AVERAGE_RATING,
ABBS.SERIES_TOTAL_VOTES,
ABBS.EPISODE_RATING,
ABBS.EPISODE_TOTAL_VOTES,
ABBS.RANK_BY_VOTES,
ABBS.RANK_BY_VOTES_ANNUALLY
FROM ALL_BEST_BY_SERIES ABBS
WHERE 1 = 1
ORDER BY ABBS.RANK_BY_VOTES
LIMIT 20;

 * postgres://postgres@localhost/imdb2
20 rows affected.


series_id,series_starting_year,series_name,series_ending_year,episode_id,episode_name,season_number,episode_number,episode_year,series_weighted_average_rating,series_total_votes,episode_rating,episode_total_votes,rank_by_votes,rank_by_votes_annually
tt0396991,2004,LazyTown,2014,tt6080222,Robbie's Dream Team,4,12,2014,5.8,292144,10.0,534,1,1
tt6111552,2015,Content Cop,None,tt6510642,Tana Mongeau,3,1,2017,9.8,40337,9.9,117,2,1
tt4517806,2015,Furusato-Time,None,tt4779920,Shingo Kazami Goes to Takaoka Part 2,1,5,2015,8.8,1848,9.9,196,3,1
tt4215716,2011,Thronecast,None,tt4691858,A Golden Crown,1,6,2011,8.1,13731,9.9,119,4,1
tt3921820,2010,Les Synaudes,None,tt5520004,Les Saisons Synaudiques,4,1,2010,9.6,3618,9.9,866,5,1
tt3911580,2009,BoOzy' OS,None,tt3921030,Interdit aux PoOlets,None,None,2009,9.5,1092,9.9,1053,6,1
tt1839578,2011,Person of Interest,2016,tt4679214,Return 0,5,13,2016,8.5,14374886,9.9,7427,7,1
tt1710308,2009,Regular Show,2017,tt6381882,A Regular Epic Final Battle,8,27,2017,8.4,5257945,9.9,344,8,2
tt0098887,1990,Parenthood,1991,tt1576815,Thanksgiving with a T That Rhymes with B That Stands for Basketball,1,10,1990,5.9,2520,9.9,160,9,1
tt6982604,2016,Alternative Lifestyle,None,tt6994224,Sugar Pine 7,1,16,2017,9.8,71508,9.8,110,10,3


# Query 2 - Movies with highest age difference between actors

This idea is inspired by youngest cast excersize.

This query will return 20 movies with highest age differences between actors.
I checked dataset and my joins and I suspect there might be dirty data in the dataset, there are people who is older than oldest people recorded in history with 122 years old(some movies included writers who is deceased in the movie starting year and I used least with death years to calculate their ages properly) and some people with negative ages.

This query uses;
 * CTE
 * Left Joins
 * Subquery
 * Multiple Window functions

In [4]:
%%sql
WITH HD AS(
SELECT 
TB.PRIMARYTITLE AS PRIMARY_TITLE,
NB.BIRTHYEAR AS BIRTH_YEAR,
NB.DEATHYEAR AS DEATH_YEAR,
MAX(NB.BIRTHYEAR) OVER (PARTITION BY TB.TCONST) AS MAX_BIRTH_YEAR,
MIN(NB.BIRTHYEAR) OVER (PARTITION BY TB.TCONST) AS MIN_BIRTH_YEAR,
TB.STARTYEAR AS START_YEAR,
NB.PRIMARYNAME AS PRIMARY_NAME,
TB.TCONST
FROM TITLE_BASICS TB
LEFT OUTER JOIN TITLE_RATINGS TR ON (TB.TCONST = TR.TCONST)
LEFT OUTER JOIN TITLE_PRINCIPALS_MELT TP ON (TB.TCONST = TP.TCONST)
LEFT OUTER JOIN NAME_BASICS NB ON (NB.NCONST = TP.PRINCIPALCAST)
WHERE 1=1
AND TR.NUMVOTES > 1000 --We used this filter to eliminate episodes which got didn't got voted significantly
AND TB.TITLETYPE <> 'tvSeries' --Filtering series
)
SELECT 
AG.PRIMARY_TITLE,
AG.START_YEAR,
AG.YOUNGEST_BIRTH_YEAR,
AG.YOUNGEST_DEATH_YEAR,
AG.YOUNGEST_PRIMARY_NAME,
AG.YOUNGEST_AGE,
AG.OLDEST_BIRTH_YEAR,
AG.OLDEST_DEATH_YEAR,
AG.OLDEST_PRIMARY_NAME,
AG.OLDEST_AGE,
(AG.OLDEST_AGE - AG.YOUNGEST_AGE) AS AGE_DIFFERENCE
FROM 
(
SELECT
YC.PRIMARY_TITLE,
YC.START_YEAR,
YC.BIRTH_YEAR AS YOUNGEST_BIRTH_YEAR,
YC.DEATH_YEAR AS YOUNGEST_DEATH_YEAR,
YC.PRIMARY_NAME AS YOUNGEST_PRIMARY_NAME,
LEAST(YC.DEATH_YEAR, YC.START_YEAR) - YC.BIRTH_YEAR AS YOUNGEST_AGE,
OC.BIRTH_YEAR AS OLDEST_BIRTH_YEAR,
OC.DEATH_YEAR AS OLDEST_DEATH_YEAR,
OC.PRIMARY_NAME AS OLDEST_PRIMARY_NAME,
LEAST(OC.DEATH_YEAR, OC.START_YEAR) - OC.BIRTH_YEAR AS OLDEST_AGE
FROM HD YC
INNER JOIN HD OC ON (YC.TCONST = OC.TCONST)
WHERE 1=1
AND YC.BIRTH_YEAR = YC.MAX_BIRTH_YEAR
AND OC.BIRTH_YEAR = OC.MIN_BIRTH_YEAR
)
AG
WHERE 1=1
AND AG.YOUNGEST_AGE >= 0
AND AG.OLDEST_AGE <= 122 --Oldest person recorded in history
ORDER BY (AG.OLDEST_AGE - AG.YOUNGEST_AGE) DESC
LIMIT 20;

 * postgres://postgres@localhost/imdb2
20 rows affected.


primary_title,start_year,youngest_birth_year,youngest_death_year,youngest_primary_name,youngest_age,oldest_birth_year,oldest_death_year,oldest_primary_name,oldest_age,age_difference
An Occurrence at Owl Creek Bridge,1964,1931,2001,Robert Enrico,33,1842,None,Ambrose Bierce,122,89
Occurrence at Owl Creek Bridge,1962,1931,2001,Robert Enrico,31,1842,None,Ambrose Bierce,120,89
Mr. Peabody & Sherman,2014,2003,None,Max Charles,11,1912,2008,Ted Key,96,85
101 Dalmatians 2: Patch's London Adventure,2003,1993,None,Bobby Lockwood,10,1896,1990,Dodie Smith,94,84
Ramona and Beezus,2010,1999,None,Joey King,11,1916,None,Beverly Cleary,94,83
L'auberge rouge,2007,1990,None,Jean-Baptiste Maunier,17,1901,2000,Claude Autant-Lara,99,82
Bag of Bones,2011,2004,None,Caitlin Carmichael,7,1922,2016,William Schallert,89,82
The Peanuts Movie,2015,2004,None,Noah Schnapp,11,1916,2008,Bill Melendez,92,81
Son of Rambow,2007,1995,None,Bill Milner,12,1914,2013,Anna Wing,93,81
Nancy Drew,2007,1991,None,Emma Roberts,16,1905,2002,Mildred Wirt Benson,97,81


# Query 3 - Best rated person for each profession
This query will return best rated person for each profession, if there is two or more people with same ratings it will pick with the latest movie starting year and if there is still two or more person it will return one based on tconst and nconst.

This query uses;
 * Transforming comma seperated values in a column to return more rows.
 * Table creation from select statement
 * Indexing for performance optimization
 * Multiple CTE's
 * Window Functions

In [5]:
%%sql
CREATE TABLE NORMALIZED_NB AS
SELECT 
NB.NCONST AS PERSON_ID,
NB.PRIMARYNAME AS PRIMARY_NAME,
NB.BIRTHYEAR AS BIRTH_YEAR,
NB.DEATHYEAR AS DEATH_YEAR,
REGEXP_SPLIT_TO_TABLE(NB.PRIMARYPROFESSION, E',') AS PROFESSION --using this to normalize the table
FROM NAME_BASICS NB;

 * postgres://postgres@localhost/imdb2
10606458 rows affected.


[]

In [6]:
%%sql
CREATE INDEX "IDX_NORMALIZED_NB#01" 
    ON NORMALIZED_NB USING BTREE(
        PERSON_ID ASC NULLS LAST
    )

 * postgres://postgres@localhost/imdb2
Done.


[]

In [19]:
%%sql
WITH MAIN_DATA AS(
SELECT
NNB.PERSON_ID,
NNB.PROFESSION,
NNB.PRIMARY_NAME,
NNB.BIRTH_YEAR,
NNB.DEATH_YEAR,
TR.TCONST,
TR.AVERAGERATING AS AVERAGE_RATING,
MAX(TR.AVERAGERATING) OVER (PARTITION BY NNB.PROFESSION) AS BEST_RATE_BY_PROFESSION
FROM NORMALIZED_NB NNB
INNER JOIN TITLE_PRINCIPALS_MELT TPM ON (TPM.PRINCIPALCAST = NNB.PERSON_ID)
INNER JOIN TITLE_RATINGS TR ON (TR.TCONST = TPM.TCONST)
WHERE 1=1
AND TR.NUMVOTES > 1000 --We used this filter to eliminate episodes which got didn't got voted significantly
AND TR.AVERAGERATING > 8 --We will check best by rating for each profession so I expect all of them greater than 8 and this will increase our performance
),
PROFESSION_TITLE AS(
SELECT 
MD.*,
TB.PRIMARYTITLE AS PRIMARY_TITLE,
TB.STARTYEAR AS START_YEAR,
TB.ENDYEAR AS END_YEAR,
TB.GENRES,
RANK() OVER (PARTITION BY MD.PROFESSION ORDER BY TB.STARTYEAR DESC, MD.TCONST DESC, MD.PERSON_ID) AS RANK_BY_PROFESSION --We are going to use this to return just one row for each profession
FROM MAIN_DATA MD
LEFT OUTER JOIN TITLE_BASICS TB ON (MD.TCONST = TB.TCONST)
WHERE 1=1
AND MD.AVERAGE_RATING = MD.BEST_RATE_BY_PROFESSION --Just returning best rated for each profession
)
SELECT
PT.PROFESSION,
PT.PRIMARY_NAME,
PT.BIRTH_YEAR,
PT.DEATH_YEAR,
PT.PRIMARY_TITLE,
PT.AVERAGE_RATING,
PT.START_YEAR,
PT.END_YEAR,
PT.GENRES
FROM PROFESSION_TITLE PT
WHERE 1=1
AND PT.RANK_BY_PROFESSION = 1
ORDER BY PT.AVERAGE_RATING DESC
LIMIT 20;

 * postgres://postgres@localhost/imdb2
20 rows affected.


profession,primary_name,birth_year,death_year,primary_title,average_rating,start_year,end_year,genres
actor,Jeff Bryan Davis,1973,None,The Ricklantis Mixup,9.9,2017,None,"Adventure,Animation,Comedy"
miscellaneous,Sarah Carbiener,None,None,The Ricklantis Mixup,9.9,2017,None,"Adventure,Animation,Comedy"
actress,Sarah Chalke,1976,None,The Ricklantis Mixup,9.9,2017,None,"Adventure,Animation,Comedy"
animation_department,Dominic Polcino,1964,None,The Ricklantis Mixup,9.9,2017,None,"Adventure,Animation,Comedy"
art_department,Miguel Sapochnik,None,None,The Winds of Winter,9.9,2016,None,"Adventure,Drama,Fantasy"
writer,Jeff Bryan Davis,1973,None,The Ricklantis Mixup,9.9,2017,None,"Adventure,Animation,Comedy"
soundtrack,Sarah Chalke,1976,None,The Ricklantis Mixup,9.9,2017,None,"Adventure,Animation,Comedy"
assistant_director,Dominic Polcino,1964,None,The Ricklantis Mixup,9.9,2017,None,"Adventure,Animation,Comedy"
camera_department,Arthur Albert,1946,None,Felina,9.9,2013,None,"Crime,Drama,Thriller"
sound_department,Ken Arsyn,None,None,Interdit aux PoOlets,9.9,2009,None,"Action,Adventure,Animation"
